In [10]:
%load_ext autoreload
%autoreload 2

from api.yelp.yelp import YelpAPI
from algo import *
from utils import *
from constraints import *
from plan import *
from fetch import *
import datetime
import time
import pytz
import dateutil.parser

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
CAT_FILE = 'api/yelp/yelpcategories.json'
with open('api/yelp/yelpkey.txt', 'r') as f:
    API_KEY = f.read().strip()
api = YelpAPI(API_KEY, CAT_FILE)

In [8]:
mex = api.business_search(location="97 5th Avenue, New York, NY 10003", 
                          categories=['mexican'], 
                          add_parent_categories=True, max_num_businesses=24)

cafe = api.business_search(location="97 5th Avenue, New York, NY 10003", 
                           categories=['cafes'], 
                           add_parent_categories=True)

movies = api.business_search(location="97 5th Avenue, New York, NY 10003", 
                             categories=['movietheaters'], 
                             add_parent_categories=True)

In [15]:
addr = '97 5th avenue, new york, ny 10003'
businesses = fetch_businesses_by_categories(api=api, 
                                            location=addr, 
                                            categories=["mexican", "cafes"],
                                            num_businesses_per_category=20)
[b.name for b in businesses]

['Los Tacos No.1',
 'Rosa Mexicano',
 'Dos Toros Taqueria',
 'Cafe Habana',
 'DORADO Tacos & Quesadillas',
 'Los Mariscos',
 'Ofrenda',
 'Cosme',
 'Agave',
 'Miscelanea NY',
 'Tacombi - Flatiron',
 'Dos Caminos',
 'Taco Mahal',
 "Benny's Burritos",
 "Otto's Tacos",
 'Flip Sigi',
 'Tortaria',
 'Javelina',
 'La Contenta',
 'Hotel Tortuga',
 'Eataly',
 'Breads Bakery',
 'Union Square Cafe',
 'Daily Provisions',
 'Brooklyn Bagel & Coffee Company',
 'La Lanterna Di Vittorio',
 'Café Grumpy',
 'The Bluebell Cafe',
 'Bao Bao Cafe',
 'Chobani SoHo',
 'Housing Works Bookstore Café',
 'Cafe Hanamizuki',
 'Maison Kayser',
 'Pasticceria Rocco',
 'Le Baratin',
 'Patisserie Claude',
 'Birch Coffee',
 'Outro',
 'Ninth Street Espresso',
 'Three Seat Espresso']

In [4]:
events = api.event_search(location="97 5th Avenue New York NY 10003", start_date="2019-03-01", end_date="2019-03-04", timezone=TIMEZONES.Eastern)

['jersey-city-jersey-city-wine-fest-3', 'manhattan-nyc-winter-tippler', 'new-york-free-friday-night-comedy-show']
2019-03-01T19:00:00-05:00 2019-03-01T22:30:00-05:00
2019-02-01T16:30:00-05:00 2019-03-02T23:30:00-05:00
2019-03-01T19:30:00-05:00 2019-03-29T21:00:00-04:00
2019-03-01T19:00:00-05:00


In [5]:
i = 2
events[i].name, events[i].start_time, events[i].end_time, events[i].url

('Camp Fairs',
 datetime.datetime(2019, 3, 3, 11, 0),
 datetime.datetime(2019, 3, 3, 13, 0),
 'https://www.yelp.com/events/long-island-city-camp-fairs?adjust_creative=UD_Or98loVGYShHgiHFW2Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_event_search&utm_source=UD_Or98loVGYShHgiHFW2Q')

In [6]:
movies['businesses'][1].categories

{'arts', 'movietheaters'}

In [9]:
[b.name for b in mex['businesses']]

['Eataly',
 'Breads Bakery',
 'Los Tacos No.1',
 'Union Square Cafe',
 'Rosa Mexicano',
 'Dos Toros Taqueria',
 'Cafe Habana',
 'Daily Provisions',
 'Brooklyn Bagel & Coffee Company',
 'La Lanterna Di Vittorio',
 'DORADO Tacos & Quesadillas',
 'Los Mariscos',
 'Ofrenda',
 'Cosme',
 'Café Grumpy',
 'The Bluebell Cafe',
 'Agave',
 'Miscelanea NY',
 'Bao Bao Cafe',
 'Tacombi - Flatiron',
 'Dos Caminos',
 'Chobani SoHo',
 'Taco Mahal',
 "Benny's Burritos"]

In [7]:
for b in cafe['businesses']:
    print(b.name, b.distance, b.rating)

Eataly 488.0897963313967 4.0
Breads Bakery 143.50184336250797 4.5
Union Square Cafe 338.7121222183838 4.0
Daily Provisions 360.226374766106 4.5
Brooklyn Bagel & Coffee Company 1041.7312658071687 4.0
La Lanterna Di Vittorio 1059.0680563302228 4.0
Café Grumpy 755.5983617867082 4.0
The Bluebell Cafe 744.9626788474335 4.0
Bao Bao Cafe 778.8511501315928 4.0
Chobani SoHo 1560.9027955233576 4.5
Housing Works Bookstore Café 1529.2163434449587 4.5
Cafe Hanamizuki 1063.000303549232 4.0
Maison Kayser 306.0148063621218 3.5
Pasticceria Rocco 1197.2549664283115 4.0
Le Baratin 743.0116623539029 4.0
Patisserie Claude 1017.6984045553193 4.0
Birch Coffee 704.3867642065566 4.0
Outro 572.4780219339871 4.0
Ninth Street Espresso 1289.5219751871368 4.0
Three Seat Espresso 1420.3683776890748 5.0


In [8]:
final_businesses = dedupe_list(mex['businesses'] + cafe['businesses'] + movies['businesses'], lambda x: x.id)
api.add_hours_to_search_results(final_businesses)
#details = []
#for biz in final_businesses:
#    details.append(api.business_details(biz.id))
#    time.sleep(0.1)


In [9]:
entities = []
for detail in final_businesses:
    entities.append(Entity(business=detail))
#for event in events:
#    entities.append(Entity(event=event))

In [10]:
seq = None # [["cafes", "mexican", "movietheaters"], ["cafes", "mexican", "movietheaters"]]
G = build_entity_graph(entities, seq, min_rating=3.0, max_distance=2000, min_dollar_signs=None, max_dollar_signs=None)

In [11]:
len(cafe['businesses']), len(mex['businesses']), len(final_businesses)

(20, 20, 60)

In [12]:
def print_edge(G, nodeA, nodeB):
    edge = G.get_edge_properties(nodeA, nodeB)
    priceA = nodeA.entity.price if nodeA.type == EntityTypes.BUSINESS else "N/A"
    priceB = nodeB.entity.price if nodeB.type == EntityTypes.BUSINESS else "N/A"
    print("{0} ({1}) --> {2} ({3})  ({4})".format(nodeA.entity.name, priceA, nodeB.entity.name, priceB, edge['dist']))

In [13]:
for n in G.nodes:
    edges = G.get_edges(n)
    for n2 in edges:
        print_edge(G, n, n2)

Three Seat Espresso ($$) --> La Lanterna Di Vittorio ($$)  (1503.6408149183544)
Three Seat Espresso ($$) --> Maison Kayser ($$)  (1550.5482666557627)
Three Seat Espresso ($$) --> Birch Coffee ($)  (1911.9889315548626)
Three Seat Espresso ($$) --> The Bluebell Cafe ($$)  (1267.2914177076723)
Three Seat Espresso ($$) --> The IFC Center (None)  (1645.0752758191275)
Three Seat Espresso ($$) --> Minetta Lane Theatre (None)  (1578.42776383347)
Three Seat Espresso ($$) --> AMC 19th St. East 6 (None)  (1397.6892039607)
Three Seat Espresso ($$) --> Chobani SoHo ($)  (1531.9145537552477)
Three Seat Espresso ($$) --> Le Baratin ($$)  (1676.8930954969337)
Three Seat Espresso ($$) --> Village East Cinema (None)  (522.9283033737584)
Three Seat Espresso ($$) --> Bao Bao Cafe ($)  (1532.2586811282965)
Three Seat Espresso ($$) --> Barrow Street Theatre (None)  (1794.0640465464319)
Three Seat Espresso ($$) --> Quad Cinema (None)  (1478.7164225470697)
Three Seat Espresso ($$) --> Film Forum (None)  (1836

In [17]:
u = UberConstraints()
b = BooleanConstraints()
se = StateExtractors()
sc = StateConstraints()
ee = EdgeExtractors()
seq_constraint = b.bool_and(
    [
        b.bool_less_than(se.get_attr("num_prev_states"), se.const(3)),
        u.follows_cat_sequence([set([x]) for x in ["mexican", "cafes", "mexican"]]),
        b.bool_less_than(ee.get_distance(use_miles=True), se.const(0.2)),
        sc.no_repeat_visits()
    ]
)

def time_spent_fn(entity):
    if "movietheaters" in entity.entity.categories:
        return [datetime.timedelta(hours=3)]
    return [datetime.timedelta(hours=1.5)]

done_constraint = b.bool_equals(se.get_attr("num_prev_states"), se.const(2))
init_constraint = sc.curr_state_category_in(set(["mexican"]))

date_start = datetime.datetime(2019, 1, 4, 12, 30)
initial_states = build_initial_states(G, [date_start], init_constraint)

neighbor_fn = build_neighbor_state_fn(seq_constraint, time_spent_fn=time_spent_fn)

success_fn = build_success_state_fn(done_constraint)

In [18]:
results = list(generate_plans(G, initial_states, neighbor_fn, success_fn, None))
results

In [21]:
def print_results(results):
    # t_format = "%m/%d/%y %B %Y %H:%M"
    t_format = "%I:%M %p"
    for state in results:
        all_prev = state.prev_state_list()
        out = []
        for s in (all_prev + [state]):
            out.append("{} ({} - {})".format(s.node.entity.name, s.start_dt.strftime(t_format), s.end_dt.strftime(t_format)))
        print(" => ".join(out))
        print()
print_results(results)

Tortaria (12:30 PM - 02:00 PM) => Breads Bakery (02:03 PM - 03:33 PM) => DORADO Tacos & Quesadillas (03:37 PM - 05:07 PM)

Rosa Mexicano (12:30 PM - 02:00 PM) => Union Square Cafe (02:03 PM - 03:33 PM) => Javelina (03:34 PM - 05:04 PM)

DORADO Tacos & Quesadillas (12:30 PM - 02:00 PM) => Breads Bakery (02:03 PM - 03:33 PM) => Tortaria (03:37 PM - 05:07 PM)

Rosa Mexicano (12:30 PM - 02:00 PM) => Daily Provisions (02:03 PM - 03:33 PM) => Javelina (03:34 PM - 05:04 PM)

DORADO Tacos & Quesadillas (12:30 PM - 02:00 PM) => Breads Bakery (02:03 PM - 03:33 PM) => Rosa Mexicano (03:36 PM - 05:06 PM)

DORADO Tacos & Quesadillas (12:30 PM - 02:00 PM) => Outro (02:02 PM - 03:32 PM) => Tortaria (03:35 PM - 05:05 PM)

Cosme (12:30 PM - 02:00 PM) => Union Square Cafe (02:02 PM - 03:32 PM) => Javelina (03:33 PM - 05:03 PM)

Cosme (12:30 PM - 02:00 PM) => Daily Provisions (02:02 PM - 03:32 PM) => Rosa Mexicano (03:36 PM - 05:06 PM)

Tortaria (12:30 PM - 02:00 PM) => Outro (02:02 PM - 03:32 PM) => Dos

In [ ]:
state = results[0]
all_states = state.prev_state_list() + [state]
print(G.get_edge_properties(all_states[1].node, all_states[2].node))

In [ ]:
results

In [22]:
initial_names = set([state.node.entity.name for state in initial])
cafe_names = set([node.name for node in cafe['businesses']])

NameError: name 'initial' is not defined

In [142]:
print(len(initial_names), len(cafe_names))
print()
print(initial_names)
print()
print(cafe_names.difference(initial_names))

2 20

{'La Lanterna Di Vittorio', 'Pasticceria Rocco'}

{'Daily Provisions', 'Breads Bakery', 'Eataly', 'Ninth Street Espresso', 'Birch Coffee', 'Le Baratin', 'Maison Kayser', 'Outro', 'Chobani SoHo', 'The Bluebell Cafe', 'Bao Bao Cafe', 'Union Square Cafe', 'Housing Works Bookstore Café', 'The Commons Chelsea', 'Cafe Hanamizuki', 'Patisserie Claude', 'Café Grumpy', 'Brooklyn Bagel & Coffee Company'}


In [127]:
for c in cafe['businesses']:
    print(c.name, c.id)

Eataly a0IET3_yCFcO36OqGSsisg
Breads Bakery mU23NlQfn1R9OhA6WMe_xg
Union Square Cafe lXGCo-K7eftxRT_6E59-Iw
Daily Provisions t4M1VX4n34eEgUWMJ7cgKw
La Lanterna Di Vittorio V-FCEicobl5Su-R7EVJQNA
Brooklyn Bagel & Coffee Company pb_ryc8FYHpUPV56eV2Eyg
Café Grumpy IYDE89R_ETmVP1-e82Y_xQ
The Bluebell Cafe 63nYipWuA42RZmESBMUNkQ
Bao Bao Cafe ynIx549QFj29o84y1RSGyw
Chobani SoHo z9sRnN1gPyxz9SJ52XDWoQ
Maison Kayser bhO25n-KN55UrqSOAb9CUQ
Housing Works Bookstore Café O4UeWRZLKuS3GRC4ELJRog
Cafe Hanamizuki S-619sO8qSxQcis0DBHvNw
Pasticceria Rocco JWLRT0dkdaC7AHtYapWbGw
Le Baratin WoZnkXZInggzgQM_LbDpGg
Patisserie Claude fvDZVAvv_g4L-WWkbDpCXg
Birch Coffee cvETUmgjYfjcAgl54aWntA
Outro IiLLntICUim7YREkIBU7lA
Ninth Street Espresso JqIrbpx-gysImU1vUgULPw
The Commons Chelsea 9BM1-Obd5Dr8pErgyBbQAQ


<function algo.build_neighbor_state_fn.<locals>.neighbor_state_fn>

In [3]:
import dateutil.parser
dateutil.parser.parse('2017-08-19T04:00:00+00:00')

datetime.datetime(2017, 8, 19, 4, 0, tzinfo=tzutc())

In [313]:
x = time.strptime('1225', "%H%M")
datetime.time(x.tm_hour, x.tm_min)

datetime.datetime(2006, 11, 21, 16, 30).time() > datetime.time(x.tm_hour, x.tm_min)

True

In [24]:
from datetime import timezone
import dateutil.parser

In [25]:
dateutil.parser.parse("2018-10-11").replace(tzinfo=timezone.utc).astimezone(tz=None)

datetime.datetime(2018, 10, 10, 20, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000), 'Eastern Daylight Time'))

In [18]:
dateutil.parser.parse("2010-03-22 04:21:01 -04:00")

datetime.datetime(2010, 3, 22, 4, 21, 1, tzinfo=tzoffset(None, -14400))

In [15]:
datetime.timezone.tzname("EST")

TypeError: descriptor 'tzname' requires a 'datetime.timezone' object but received a 'str'

ModuleNotFoundError: No module named 'tzwhere'

In [46]:
dt = datetime.datetime.now() # dateutil.parser.parse("04:21AM 12-22-2018")
print(pytz.timezone(TIMEZONES.Eastern).localize(dt).timestamp())
print(pytz.timezone(TIMEZONES.Pacific).localize(dt).timestamp())

1546227495.213787
1546238295.213787


In [248]:
y = 4
3 < y < 7

True

In [249]:
z = datetime.timedelta(seconds=1800)

In [250]:
datetime.datetime(2006, 11, 21, 16, 30) + z

datetime.datetime(2006, 11, 21, 17, 0)

In [78]:
dt = datetime.datetime.now()

In [85]:
a = pytz.timezone(TIMEZONES.Eastern).localize(dt)
b = pytz.timezone(TIMEZONES.Pacific).localize(a.replace(tzinfo=None))
a, b

(datetime.datetime(2018, 12, 30, 23, 1, 45, 950477, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
 datetime.datetime(2018, 12, 30, 23, 1, 45, 950477, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>))

In [142]:
txt = """
                    distance=biz.get('distance'),
                    transactions=biz.get('transactions'),
"""
z = "\n".join([t.split('=')[0] for t in txt.split()])
print(z)

distance
transactions
